* [Issue 2](https://github.com/salgo60/Dogpark_Sweden/issues/2)
* denna notebook [Dogpark_Sweden.ipynb]()

In [2]:
import requests, json, time
from bs4 import BeautifulSoup

BASE_URL = "https://www.hundlistan.se/"
params = {
    "mylisting-ajax": "1",
    "action": "get_listings",
    "security": "5e528da4d5",  # säkerhets-token (kan behöva uppdateras om Hundlistan ändrar den)
    "form_data[preserve_page]": "true",
    "form_data[sort]": "latest",
    "form_data[region]": "",
    "form_data[search_location]": "",
    "form_data[lat]": "false",
    "form_data[lng]": "false",
    "listing_type": "hundpark",
    "listing_wrap": "col-md-12 grid-item"
}

features = []

for page in range(1, 40):  # 1 till 39
    params["form_data[page]"] = str(page)
    print(f"Hämtar sida {page}/39 ...", end="\r")

    r = requests.get(BASE_URL, params=params)
    if r.status_code != 200:
        print(f"\n⚠️ Sida {page} misslyckades ({r.status_code})")
        continue

    data = r.json()
    html = data.get("html", "")
    soup = BeautifulSoup(html, "html.parser")

    for item in soup.select(".lf-item-container"):
        try:
            data_id = item.get("data-id")
            title = item.select_one(".listing-preview-title").get_text(strip=True)
            href = item.select_one("a")["href"]
            lan = item.select_one(".lf-contact li").get_text(strip=True)
            loc = json.loads(item.get("data-locations"))[0]

            address = loc.get("address")
            lat = float(loc.get("lat"))
            lng = float(loc.get("lng"))

            popup_html = f"""
            <b>{title}</b><br>
            {address}<br>
            {lan}<br>
            <a href="{href}" target="_blank">Visa på Hundlistan</a>
            """.strip()

            features.append({
                "type": "Feature",
                "geometry": {"type": "Point", "coordinates": [lng, lat]},
                "properties": {
                    "id": data_id,
                    "namn": title,
                    "adress": address,
                    "lan": lan,
                    "url": href,
                    "popup": popup_html
                }
            })
        except Exception as e:
            print(f"\n⚠️ Fel vid parsing: {e}")

    time.sleep(0.5)  # vänlig delay mot servern

geojson = {
    "type": "FeatureCollection",
    "features": features
}

with open("hundparker_umap.geojson", "w", encoding="utf-8") as f:
    json.dump(geojson, f, ensure_ascii=False, indent=2)

print(f"\n✅ Klart! {len(features)} hundparker sparade i hundparker_umap.geojson")


Hämtar sida 39/39 ...
✅ Klart! 760 hundparker sparade i hundparker_umap.geojson
